#  Data Cleaning Practice

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import requests

os.chdir('/Users/sundaswiqas/Documents/RawData')
salaries = pd.read_csv('2020 Female Salaries in Tech - Form Responses 1.csv',skiprows=5)


salaries.rename(columns={'Unnamed: 0': 'date',
                         'This sheet shares title & salary info submitted by women in tech. This sheet is publically available to ALL women who could use it in salary or raise negotiations!':'title',
                         'Unnamed: 2': 'salary',
                         'Unnamed: 3': 'location',
                         'Unnamed: 4': 'benefits',
                         'Unnamed: 5': 'yearsofexperience'}, inplace=True)

salaries.drop([0,1], axis=0, inplace=True)
salaries.dropna(axis=0, how='all', inplace=True)
salaries.drop('date', axis=1, inplace=True)

for col in ['title','benefits','location','yearsofexperience','salary']:
    salaries[col] = salaries[col].str.lower()
    
def money_to_float(salary):
    try:
        if re.search('^\d+$', salary):
            return float(salary)
        elif re.search('^\$(\d{1,3})(\,\d{3})*|(\d+)(\.\d{2})?$', salary):
            salary = salary.replace(',','').replace('!','')
            salary = re.sub('\$|€|£','',salary)
            salary = re.sub(' ','',salary)
            return float(salary)
        elif re.search('^\d+[k]|^\$(\d+)[k]', salary):
            salary = salary.replace(',','').replace('£','').replace('$','')
            salary = salary.replace('k', '')
            salary = float(salary)*1000
            return salary
        else:
            return 'unconvertible'
    except:
        pass
    
def fix_salaries_with_text(salary):
    
    salary = re.sub('[(),$]+','',salary)

    try:
        if re.search('[a-zA-Z]+', salary):

            if re.search('^\d+|^\d+[k]|^(\d+) *[a-zA-Z]+$|^[a-zA-Z]+ *\d+', salary) and not re.search('%', salary):
                
                if re.search('^\d+[k]',salary):
                    salary = salary.replace('k', '')
                    salary = re.findall('\d+', salary)
                    salary = [float(salary) for salary in salary]
                    salary = sum(salary)
                    salary = salary*1000
                else:
                    salary = re.findall('\d+', salary)
                    salary = [float(salary) for salary in salary]
                    salary = sum(salary)
                return salary


            elif re.search('%',salary):
                base = float(re.findall('\d+', salary)[0])
                percent = float(re.findall('\d+', salary)[-1]) / 100
                salary = base + (base*percent)
                return salary
    except:
        return 'unconvertible'
    
mask = ~salaries.salary.str.contains('[a-zA-Z]+', regex=True)
salaries_notext = salaries.loc[mask] 
salaries_notext['salary_temp'] = salaries_notext.loc[:,'salary'].apply(money_to_float)

mask = salaries.salary.str.contains('[a-zA-Z]+', regex=True)
salaries_withtext = salaries.loc[mask]
salaries_withtext['salary_temp'] = salaries_withtext['salary'].apply(fix_salaries_with_text)

salaries = pd.concat([salaries_notext, salaries_withtext], axis=0, ignore_index=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
currencies = ['CAD', 'Canadian', '£', 'CDN', '€', 'AUD', 'SGD', 'AU', 'GBP', 'kr/år', 'INR',
'SEK', 'EUROS', 'DKK']
monies = '|'.join([c for c in currencies])

currency_dict = {'Canadian': 'CAD',
                 'CDN': 'CAD',
                 'EUROS':'EUR',
                 '€':'EUR',
                 '£': 'GBP',
                 'AU': 'AUD',
                 'kr/år': 'SEK'}

def get_currency(string):

        if any(currency in string.upper() for currency in currencies):
            currency = re.findall(monies,string.upper())[0]
            return currency
        else:
            return 'USD'
        
def currency_converter(currency, salary):
    if currency == 'USD':
        return salary
    else:
        exrate = rates[currency]
        salary = salary / exrate
        print(salary)
        return salary
    
salaries.dropna(how='all',inplace=True)
salaries['currency'] = salaries['salary'].apply(get_currency)
salaries['currency'] = salaries['currency'].replace(currency_dict)

# Where USD is the base currency you want to use
url = 'https://api.exchangerate-api.com/v4/latest/USD'
response = requests.get(url)
data = response.json()
rates = data['rates']

salaries['salary_temp'] = salaries.apply(lambda x: currency_converter(x['currency'],x['salary_temp']) if x['salary_temp'] != 'unconvertible' else np.nan,axis=1)

106604.18539253344
57076.53963965678
31644.61082178361
79907.15549551949
95127.56606609463
59613.27473475264
43441.58850351655
34119.08702903928
56107.465996070234
101786.49569072126
60247.4585085266
46929.59925927335
40958.45017713127
82443.89059061535
48197.966806821285
57076.53963965678
65084.66055544147
151490.15818938962
88785.72832835499
73565.31775777985
41412.200427439864
158545.94344349107
61718.21259567726
84161.19899410535
57076.53963965678
21562.248308314785
31709.18868869821
44885.972796856186
53863.16735622742
101469.40380383428
63418.37737739642
107.81124154157392
88785.72832835499
88785.72832835499
64686.74492494435
86405.49763394815
109079.60908912185
63418.37737739642
46929.59925927335
88785.72832835499
38473.39281977132
68491.84756758813
52516.58817232174
47026.00320934385
52090.34201650396
94052.0064186877
101286.77614320214
nan
110649.5336908501
61495.542658372724
63684.73508742322
31409.31134511713
47026.00320934385
70053.20859616554
58589.95628042936
92342.865876

In [22]:
salaries.sort_values(by='salary_temp',ascending=False).tail(55)

,title,salary,location,benefits,yearsofexperience,salary_temp,currency
837,software engineer,£26900.00,"belfast, uk",pension/some private healthcare,3,34119.087029,GBP
1386,junior producer,£25000.00,sunderland,none,3,31709.188689,GBP
456,ux/ui designer,"€28,200.00","the hague, netherlands",25 vacation days,"1,5",31644.610822,EUR
1670,operations coordinator,49320aud,"melbourne, australia","part-time work, flexible hours","three years in various roles, 18 months with c...",31409.311345,AUD
1723,data scientist lead,"inr 23,00,000","bangalore, india",NaN,6,31021.690474,INR
596,jr ux designer,"28,000","sumner, wa.",no.,1-2,28000.000000,USD
1176,phd candidate,27000,"sydney, australia",NaN,3,27000.000000,USD
1722,staff solution engineer,27000 usd,"malaga, spain",no,8,27000.000000,USD
1758,level designer,"21,000 gbp","north west, england, u.k","flexi-time, healthcare schemes, remote working",2+,26635.718499,GBP
292,ux/ui designer,25000,mexico,"6 days of vacation/year, medical insurance, ho...",3,25000.000000,USD


In [3]:
normal_responses = re.compile('^\d+$|^\d\.\d+$')
years_responses = re.compile('^\d+ *years?$|^\d+\.\d+ *years?$')
young_uns = re.compile('entry|fresh|new| *grad')

def get_yoe(sent):
    sent = str(sent)
    print(sent)
    if pd.isnull(sent):
        return np.nan
    elif re.search(normal_responses, sent):
        years = re.search(normal_responses, sent)
        years = float(years.group(0))
        return years
    elif re.search('^\d\-\d+$', sent):
        years = re.findall('^\d\-\d+$', sent)
        years = re.sub('-', '', years[0]).split()
        years = sum([float(x) for x in years]) / 2
        return years
    elif re.search(years_responses, sent):
        years = re.search(years_responses, sent)
        years = years.group(0)
        years = float(re.sub('years?','', years))
        return years
    elif re.search('^\d+\-\d+ *years?$', sent):
        years = re.search('^\d+\-\d+ *years?$', sent)
        years = years.group(0)
        years = re.sub('-|years?',' ',years).split()
        years = [float(x) for x in years]
        years = sum(years) / 2
        return years
    elif re.search('^\d+ *months?$', sent):
        years = re.search('^\d+ *months?$', sent)
        years = years.group(0)
        years = re.sub('months', '', years)
        years = float(years) / 12
        return years
    elif re.findall('\d+ *[a-zA-Z]+[experience]|[experience] *\d+', sent):
        years_list = re.findall('\d+', sent)
        years = max([float(year) for year in years_list])
        return years
    elif re.search('\d+ *total', sent):
        years = re.findall('\d+ *total', sent)[0]
        years = re.sub('total','', years).strip()
        years = float(years)
        return years
    elif re.findall('\d+ *in [a-zA-Z]+', sent):
        years = re.findall('\d+ *in [a-zA-Z]+', sent)
        years = sum([float(x) for x in years])
        return years
    elif re.search(young_uns, sent):
        years = 0
        return years

salaries['yoe'] = salaries['yearsofexperience'].apply(get_yoe)

10
10
nan
nan
1
2
5
nan
nan
nan
nan
nan
nan
nan
nan
nan
4 years
nan
nan
nan
4
nan
1 year
nan
nan
nan
nan
nan
nan
nan
nan
5 years out of a bootcamp
nan
2
nan
nan
nan
nan
6
nan
nan
2
nan
16
nan
nan
nan
nan
nan
20
nan
15
nan
nan
1
nan
nan
nan
2
i am 20k under priced
nan
0 yrs cloud data engineering exp, 8 years etl experience
1.5
nan
15
nan
nan
6
5
nan
12
11
nan
nan
nan
nan
4
nan
nan
1
2
nan
nan
nan
nan
nan
nan
nan
10
nan
nan
nan
15
1
9
8
3
2
nan
nan
nan
15
1
nan
nan
nan
nan
3 years
nan
nan
nan
2.5
nan
nan
nan
nan
1
3
nan
nan
entry level with 2 internships
4
14
15+
2
7
5
6
nan
<1
1
9
15
9
11
7
5 in product; 15 more in product marketing/communications
first job 
9
8
0
new grad
1
10 years work experience, 3 years analyst experience
6 years tech, 1 in this role
3
3
3
6
4
4 years
12
10+ years
3-4
10
3
10+
nan
2
2
5 years
3
10
13
1
nan
nan
20
2
9
8
8
1 year in ux and 12 years in graphic design
18
6
8
4
6 months
11
3
1 year as a software engineer, 9 years of misc. professional experience
nan
5


In [4]:
def clean_locations(location):
    if pd.isnull(location):
        return np.nan
    if 'remote' in location:
        return 'remote'
    elif 'nyc' in location or 'new york' in location:
        return 'new york city'
    elif 'sf' in location or 'san francisco' in location:
        return 'san francisco'
    elif 'san diego' in location:
        return 'san diego'
    elif 'san jose' in location:
        return 'san jose'
    elif 'san mateo' in location:
        return 'san mateo'
    elif 'san ramon' in location:
        return 'san ramon'
    elif 'santa monica' in location:
        return 'san monica'
    elif 'palo alto' in location:
        return 'palo alto'
    elif 'mountain view' in location:
        return 'mountain view'
    elif '-' in location or ',' in location:
        location = re.split(',|-', location)[0].strip()
        return location
    else:
        return location

salaries['location'] = salaries['location'].apply(clean_locations)

In [5]:
salaries['401k'] = salaries.benefits.apply(lambda x: 1 if re.search('(?<!no )401k', str(x).lower())else 0)
salaries['unlimited_pto'] = salaries.benefits.apply(lambda x: 1 if re.search('(?<!no )unlimited pto' ,str(x).lower()) else 0)
salaries['equity'] = salaries.benefits.apply(lambda x: 1 if re.search('(?<!no )equity' , str(x).lower()) else 0)
salaries['healthcare'] = salaries.benefits.apply(lambda x: 1 if re.search('(?<!no )healthcare|(?<!no )insurance', str(x).lower()) else 0)

In [6]:
def job_area(title):
    if re.search('data|business intelligence',title):
        return 'data'
    elif re.search('engineer|developer|design|ux',title):
        return 'tech'
    elif re.search('marketing|communications|growth|writer|brand',title):
        return 'marketing'
    elif re.search('sales|account manager',title):
        return 'sales'
    elif re.search('product',title):
        return 'product'
    elif re.search('talent|human resources|hr|recruit',title):
        return 'hr'
    elif re.search('law|legal|counsel',title):
        return 'legal'
    elif re.search('strategy|business|finance',title):
        return 'strategy and ops'
    else:
        return 'other'
    
salaries['job_area'] = salaries.title.apply(job_area)

In [7]:
#salaries = salaries.drop(['salary', 'benefits'], axis=1)
#salaries.rename(columns={'salary_temp':'salary'}, inplace=True)

In [8]:
# s = salaries.loc[salaries.salary != 'unconvertible']
cols = ['salary', 'salary_temp']
s = salaries.loc[~salaries.salary.isna(), 'salary']
s = s.apply(lambda s: float(s))